In [134]:
import numpy as np
import pandas as pd
from arch import arch_model
from statsmodels.tsa.stattools import adfuller
import pandas as pd

df = pd.read_csv('CryptoTableauData.csv')
df.head(10)
df_bitcoin = df[df['crypto_name'] == 'Bitcoin']
df_bitcoin.head()

df_ret = pd.read_csv('CryptoReturnsData.csv')
df_ret = df_ret.drop(['trade_date'], axis=1)
df_ret = df_ret.dropna()
df_ret_abs = df_ret.abs()
df_ret.head()

df_logret = pd.read_csv('CryptoLogReturnsData.csv')
df_logret = df_logret.drop(['trade_date'], axis=1)
df_logret = df_logret.dropna()
df_logret_abs = df_logret.abs()
df_logret.head()

df_vol = pd.read_csv('CryptoVolumeData.csv')
df_vol = df_vol.drop(['trade_date'], axis=1)
df_vol = df_vol.dropna()
#df_vol = df_vol.abs()
df_vol.head()

,Binance_Coin,Bitcoin,Bitcoin_Cash,Bitcoin_SV,Cardano,Chainlink,Crypto_com_Coin,EOS,Ethereum,Huobi_Token,Litecoin,Monero,NEO,Stellar,Tether,Tezos,TRON,UNUS_SED_LEO,USD_Coin,XRP
0,13595933.0,2285901.0,8260875.0,4913763.0,1.109614e+09,32612915,393131898,554819091.0,43968425.0,27610043.0,57244277.0,3110149.0,49121561.0,4.091794e+09,23973071925,11407697.0,4.278287e+10,6863967,222754025,4.959182e+09
1,12725873.0,2277170.0,8908406.0,4778668.0,9.123387e+08,39574860,383223801,528716218.0,47823167.0,27839594.0,55616064.0,4016523.0,41989830.0,4.423396e+09,24597499287,13197268.0,4.307836e+10,7477121,208342328,4.702822e+09
2,14481198.0,2806769.0,8478598.0,5295787.0,1.458810e+09,45666041,624589389,779468280.0,57224534.0,29730444.0,69298745.0,3247277.0,47475221.0,4.775886e+09,30710349697,18755269.0,4.902068e+10,9116221,310602968,5.573760e+09
3,12719992.0,2793752.0,7750431.0,5026660.0,1.892762e+09,44377600,597677920,876575966.0,53368351.0,29452557.0,75693466.0,4651344.0,45056169.0,9.623521e+09,30721581637,14756402.0,4.501190e+10,12641495,294667615,6.534276e+09
4,11626864.0,2473762.0,8063560.0,4282231.0,1.607080e+09,56364011,617465025,669064411.0,53330141.0,41762497.0,58873479.0,4072668.0,41137525.0,4.384331e+09,28578448466,26804412.0,4.993091e+10,13814759,254889570,5.857837e+09


In [129]:
def adfuller_test(series, signif=0.05):
    x = adfuller(series, autolag='AIC')

    #using dictionary saves different data types (float, int, boolean)
    output = {'Test Statistic': x[0], 
              'P-value': x[1], 
              'Number of lags': x[2], 
              'Number of observations': x[3],
              f'Reject (signif. level {signif})': x[1] < signif }

    for key, val in x[4].items():
         output[f'Critical value {key}'] = val
    
    return pd.Series(output)

In [137]:
df_logret_adf = df_logret_abs.apply(lambda x: adfuller_test(np.array(x)), axis=0)
df_vol_adf = df_vol.apply(lambda x: adfuller_test(np.array(x)), axis=0)
df_logret_adf.head()
#df_vol_adf.head()

,Binance_Coin,Bitcoin,Bitcoin_Cash,Bitcoin_SV,Cardano,Chainlink,Crypto_com_Coin,EOS,Ethereum,Huobi_Token,Litecoin,Monero,NEO,Stellar,Tether,Tezos,TRON,UNUS_SED_LEO,USD_Coin,XRP
Test Statistic,-4.495731,-4.035716,-4.942127,-5.718945,-16.572584,-16.059405,-3.286203,-7.046448,-5.312662,-6.76409,-6.719079,-4.07046,-3.888246,-15.821224,-6.757029,-7.267776,-6.577143,-7.775443,-2.244799,-5.474674
P-value,0.0002,0.001235,0.000029,0.000001,0.0,0.0,0.015507,0.0,0.000005,0.0,0.0,0.001083,0.002123,0.0,0.0,0.0,0.0,0.0,0.190389,0.000002
Number of lags,6,10,6,3,0,0,16,3,7,3,3,9,9,0,3,3,3,2,15,7
Number of observations,344,340,344,347,350,350,334,347,343,347,347,341,341,350,347,347,347,348,335,343
Reject (signif. level 0.05),True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True


In [131]:
import warnings
warnings.filterwarnings('ignore')

def archmodel(series):
    garch11 = arch_model(series, p=1, q=1)
    res = garch11.fit(update_freq=10)
    return pd.Series(res.params)

df_ret_garch = df_ret.apply(lambda x: archmodel(np.array(x)), axis=0)
df_vol_garch = df_vol.apply(lambda x: archmodel(np.array(x)), axis=0)
df_ret_garch.head()
df_vol_garch.head()

Iteration:     10,   Func. Count:     59,   Neg. LLF: 999.7108790101602
Optimization terminated successfully    (Exit mode 0)
            Current function value: 999.7108790101602
            Iterations: 10
            Function evaluations: 59
            Gradient evaluations: 10
Iteration:     10,   Func. Count:     57,   Neg. LLF: 923.5446604239692
Optimization terminated successfully    (Exit mode 0)
            Current function value: 923.5446605615932
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1043.2177241297209
            Iterations: 9
            Function evaluations: 53
            Gradient evaluations: 9
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1164.6427500722675
            Iterations: 9
            Function evaluations: 50
            Gradient evaluations: 9
Iteration:     10,   F

C:\Users\Tharun\anaconda3\lib\site-packages\arch\univariate\base.py:753: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


 5
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5759.4537199024535
            Iterations: 7
            Function evaluations: 39
            Gradient evaluations: 7
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7908.824128150794
            Iterations: 4
            Function evaluations: 23
            Gradient evaluations: 4
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6658.016000390262
            Iterations: 7
            Function evaluations: 40
            Gradient evaluations: 7
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7153.494069623934
            Iterations: 6
            Function evaluations: 33
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7391.522261896741
            Iterations: 7
            Function evaluations: 

,Binance_Coin,Bitcoin,Bitcoin_Cash,Bitcoin_SV,Cardano,Chainlink,Crypto_com_Coin,EOS,Ethereum,Huobi_Token,Litecoin,Monero,NEO,Stellar,Tether,Tezos,TRON,UNUS_SED_LEO,USD_Coin,XRP
mu,1.706841e+07,3.357075e+06,1.046805e+07,7.767213e+06,3.238903e+09,1.008612e+08,3.618217e+08,8.854662e+08,5.988077e+07,4.477863e+07,6.346015e+07,4.017658e+06,4.543402e+07,5.444360e+09,3.801935e+10,5.886949e+07,7.100250e+10,1.074713e+07,4.790034e+08,8.350316e+09
omega,9.670912e+11,5.224484e+10,9.886469e+11,1.492543e+12,8.397469e+16,2.488769e+14,3.416541e+15,1.761499e+16,1.760141e+13,1.918342e+13,1.257187e+13,4.679946e+13,7.997897e+12,1.974119e+17,4.053703e+18,2.811807e+14,1.702511e+19,1.969444e+11,4.387393e+16,7.627673e+18
alpha[1],4.191737e-01,5.026584e-01,2.917023e-01,3.511640e-01,4.399522e-01,3.900855e-01,3.212322e-01,4.114703e-01,2.796174e-01,3.336881e-01,4.725647e-01,1.327708e-01,5.146695e-01,3.356178e-01,3.635193e-01,5.160200e-01,5.197951e-01,5.537631e-01,5.510876e-01,7.593598e-01
beta[1],5.808263e-01,4.973416e-01,7.082977e-01,5.629569e-01,5.600478e-01,5.350576e-01,6.003888e-01,5.213558e-01,7.081166e-01,6.663119e-01,5.274353e-01,7.056742e-01,4.853305e-01,6.643822e-01,6.364807e-01,1.611617e-01,4.802049e-01,4.462369e-01,0.000000e+00,0.000000e+00


In [132]:
import warnings
warnings.filterwarnings('ignore')

def archmodel(series):
    tarch11 = arch_model(series, p=1, q=1, power=1.0)
    res = tarch11.fit(update_freq=10)
    return pd.Series(res.params)

df_ret_tarch = df_ret.apply(lambda x: archmodel(np.array(x)), axis=0)
df_vol_tarch = df_vol.apply(lambda x: archmodel(np.array(x)), axis=0)
df_ret_tarch.head()
df_vol_tarch.head()

Iteration:     10,   Func. Count:     59,   Neg. LLF: 1005.8742811052673
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1005.8742817686705
            Iterations: 10
            Function evaluations: 59
            Gradient evaluations: 10
Iteration:     10,   Func. Count:     61,   Neg. LLF: 927.2087592148193
Optimization terminated successfully    (Exit mode 0)
            Current function value: 927.2087595335602
            Iterations: 10
            Function evaluations: 61
            Gradient evaluations: 10
Iteration:     10,   Func. Count:     61,   Neg. LLF: 1045.1261000735212
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1045.1261000735212
            Iterations: 11
            Function evaluations: 65
            Gradient evaluations: 11
Iteration:     10,   Func. Count:     60,   Neg. LLF: 1176.2961550099863
Optimization terminated successfully    (Exit mode 0)
            Current functi

,Binance_Coin,Bitcoin,Bitcoin_Cash,Bitcoin_SV,Cardano,Chainlink,Crypto_com_Coin,EOS,Ethereum,Huobi_Token,Litecoin,Monero,NEO,Stellar,Tether,Tezos,TRON,UNUS_SED_LEO,USD_Coin,XRP
mu,1.706841e+07,3.357075e+06,1.046805e+07,7.767213e+06,3.238903e+09,1.008612e+08,3.618217e+08,8.854662e+08,5.988077e+07,4.477863e+07,6.346015e+07,4.017658e+06,4.543402e+07,5.444360e+09,3.801935e+10,5.886949e+07,7.100250e+10,1.074713e+07,4.790034e+08,8.350316e+09
omega,6.953744e+05,3.232487e+04,7.030814e+05,3.863344e+05,2.049081e+08,1.496627e+07,5.545166e+07,1.259106e+08,2.966598e+06,3.097049e+06,2.507177e+06,6.489955e+06,3.999474e+05,3.141750e+08,1.423675e+09,9.184455e+06,2.917628e+09,3.138028e+05,1.481113e+08,1.952905e+09
alpha[1],4.262097e-01,3.460936e-01,4.258131e-01,2.623702e-01,4.681705e-01,3.377158e-01,3.858324e-01,3.691168e-01,3.333871e-01,3.766288e-01,4.832545e-01,4.204960e-01,2.848740e-01,3.687447e-01,3.510808e-01,3.816789e-01,5.457858e-01,6.121644e-01,3.299655e-01,6.239705e-01
beta[1],5.265219e-01,6.539064e-01,5.648661e-01,6.726760e-01,4.749036e-01,3.971616e-01,3.621690e-01,3.617481e-01,5.865572e-01,5.917346e-01,4.653303e-01,4.733002e-01,7.151260e-01,5.640560e-01,5.889315e-01,3.551498e-01,4.100906e-01,3.878356e-01,2.295327e-01,3.889386e-11
